This file grabs everything from the LSMS survey that I think an image could possibly recognize and uses those features to predict consumption. This serves as a "gold standard" for any image-based model. It turns out that the CNN model performs almost as well as this gold standard!

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_stata('../process_data/data/input/LSMS/malawi_2016/IHS4 Consumption Aggregate.dta')

In [3]:
df_geo = pd.read_stata('../process_data/data/input/LSMS/malawi_2016/HouseholdGeovariables_stata11/HouseholdGeovariablesIHS4.dta')
df_hhf = pd.read_stata('../process_data/data/input/LSMS/malawi_2016/HH_MOD_F.dta')
df_plot = pd.read_stata('../process_data/data/input/LSMS/malawi_2016/PlotGeovariablesIHS4.dta')

In [4]:
df_com = pd.read_stata('../process_data/data/input/LSMS/malawi_2016/COM_CD.dta')
df_com2 = pd.read_stata('../process_data/data/input/LSMS/malawi_2016/COM_CF1.dta')

In [5]:
# rooms = df_hhf['hh_f10']
# roof = df_hhf['hh_f08']

# # all distance infrasturcture metrics
# road_type = df_com['com_cd01']
# dist_daily_market = df_com['com_cd16']
# dist_larger_weekly = df_com['com_cd18a']
# dist_perm_admarc = df_com['com_cd20a']
# dist_post_office = df_com['com_cd22a']
# dist_telephone = df_com['com_cd24a']
# dist_gov_prim_school = df_com['com_cd27a']
# dist_gov_sec_school = df_com['com_cd36a']
# dist_comm_sec_school = df_com['com_cd40a']
# dist_medicines = df_com['com_cd49a']
# dist_health_clinic = df_com['com_cd51a']
# dist_doctor = df_com['com_cd60a']
# dist_bank = df_com['com_cd67a']
# dist_microfinance = df_com['com_cd69a']

# dist_agric_exten_officer = df_com2['com_cf08a']

# dist_admarc_outlet = df_geo['dist_admarc']
# dist_agric_market = df_geo['dist_agmrkt']
# dist_tobacco_auction = df_geo['dist_auction']
# dist_boma = df_geo['dist_boma']
# dist_border = df_geo['dist_borderpost']
# dist_popcenter = df_geo['dist_popcenter']
# dist_road = df_geo['dist_road']

# dist_hh = df_plot['dist_hh']

# # temp
# mean_temp = df_geo['af_bio_1']
# mean_temp_wet_q = df_geo['af_bio_8']

# # rain
# mean_rain = df_geo['af_bio_12']
# mean_rain_wet_month = df_geo['af_bio_13']
# mean_rain_wet_q = df_geo['af_bio_16']

In [6]:
hhf_input = df_hhf[['case_id', 'HHID', 'hh_f10', 'hh_f08']]
com_input = df_com[['ea_id', 'com_cd01', 'com_cd16', 'com_cd18a', 'com_cd20a', 'com_cd22a', 'com_cd24a',
                   'com_cd27a', 'com_cd36a', 'com_cd40a', 'com_cd49a', 'com_cd51a', 'com_cd60a', 'com_cd67a',
                   'com_cd69a']]

com2_input = df_com2[['ea_id', 'com_cf08a']]

geo_input = df_geo[['case_id', 'HHID', 'dist_admarc', 'dist_agmrkt', 'dist_auction', 'dist_boma', 'dist_borderpost',
                  'dist_popcenter', 'dist_road', 'af_bio_1', 'af_bio_8', 'af_bio_12', 'af_bio_13', 'af_bio_16']]

plot_input = df_plot[['case_id', 'HHID', 'dist_hh']]

In [7]:
df_cons = df[['case_id', 'ea_id', 'rexpagg']]
df_cons['cons'] = df_cons['rexpagg']/365
df_cons.drop('rexpagg', axis=1, inplace=True)

/Users/jmathur25/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
df_merge = pd.merge(df_cons, hhf_input, on='case_id', how='left')
print(df_merge.shape)
df_merge = pd.merge(df_merge, com_input, on='ea_id', how='left')
print(df_merge.shape)
df_merge = pd.merge(df_merge, com2_input, on='ea_id', how='left')
print(df_merge.shape)
df_merge = pd.merge(df_merge, geo_input, on=['case_id', 'HHID'], how='left')
print(df_merge.shape)
df_merge = pd.merge(df_merge, plot_input, on=['case_id', 'HHID'], how='left')
print(df_merge.shape)

(12447, 6)
(12447, 20)
(12447, 21)
(12447, 33)
(19870, 34)


In [9]:
df_merge.head()

,case_id,ea_id,cons,HHID,hh_f10,hh_f08,com_cd01,com_cd16,com_cd18a,com_cd20a,...,dist_boma,dist_borderpost,dist_popcenter,dist_road,af_bio_1,af_bio_8,af_bio_12,af_bio_13,af_bio_16,dist_hh
0,301025230225,30102523,815.565491,0001c970eecf473099368557e2080b3e,2,GRASS,GRADED GRAVELED,10.0,10.0,10.0,...,44.0,39.0,32.0,13.0,220.0,234.0,944.0,238.0,660.0,0.2
1,210374850204,21037485,4746.132812,000509f5cfcc4b078a09672b09425e95,3,IRON SHEETS,TAR/ASPHALT,NaN,NaN,NaN,...,3.0,74.0,3.0,2.0,200.0,218.0,863.0,224.0,599.0,NaN
2,311057710075,31105771,1281.518311,000bc107780044e59327dbf7ec960ac1,3,GRASS,NaN,NaN,NaN,NaN,...,12.0,36.0,12.0,1.0,257.0,280.0,937.0,199.0,549.0,1.1
3,311057710075,31105771,1281.518311,000bc107780044e59327dbf7ec960ac1,3,GRASS,NaN,NaN,NaN,NaN,...,12.0,36.0,12.0,1.0,257.0,280.0,937.0,199.0,549.0,1.1
4,311057710075,31105771,1281.518311,000bc107780044e59327dbf7ec960ac1,3,GRASS,NaN,NaN,NaN,NaN,...,12.0,36.0,12.0,1.0,257.0,280.0,937.0,199.0,549.0,1.2


In [10]:
df_cords = df_geo[['case_id', 'HHID', 'lat_modified', 'lon_modified']]
df_final = pd.merge(df_merge, df_cords, on=['case_id', 'HHID'], how='left')

In [11]:
df_final.shape

(19870, 36)

In [12]:
df_final.head()

,case_id,ea_id,cons,HHID,hh_f10,hh_f08,com_cd01,com_cd16,com_cd18a,com_cd20a,...,dist_popcenter,dist_road,af_bio_1,af_bio_8,af_bio_12,af_bio_13,af_bio_16,dist_hh,lat_modified,lon_modified
0,301025230225,30102523,815.565491,0001c970eecf473099368557e2080b3e,2,GRASS,GRADED GRAVELED,10.0,10.0,10.0,...,32.0,13.0,220.0,234.0,944.0,238.0,660.0,0.2,-14.683761,34.915074
1,210374850204,21037485,4746.132812,000509f5cfcc4b078a09672b09425e95,3,IRON SHEETS,TAR/ASPHALT,NaN,NaN,NaN,...,3.0,2.0,200.0,218.0,863.0,224.0,599.0,NaN,-14.005029,33.794591
2,311057710075,31105771,1281.518311,000bc107780044e59327dbf7ec960ac1,3,GRASS,NaN,NaN,NaN,NaN,...,12.0,1.0,257.0,280.0,937.0,199.0,549.0,1.1,-16.826165,35.269503
3,311057710075,31105771,1281.518311,000bc107780044e59327dbf7ec960ac1,3,GRASS,NaN,NaN,NaN,NaN,...,12.0,1.0,257.0,280.0,937.0,199.0,549.0,1.1,-16.826165,35.269503
4,311057710075,31105771,1281.518311,000bc107780044e59327dbf7ec960ac1,3,GRASS,NaN,NaN,NaN,NaN,...,12.0,1.0,257.0,280.0,937.0,199.0,549.0,1.2,-16.826165,35.269503


In [13]:
df_use = df_final.drop(['case_id', 'ea_id', 'HHID'], axis=1)

In [14]:
df_use = pd.get_dummies(df_use)

In [15]:
clusters = df_use.groupby(['lat_modified', 'lon_modified'])

In [16]:
cluster_df = clusters.mean().reset_index()

In [17]:
cluster_df

,lat_modified,lon_modified,cons,hh_f10,com_cd16,com_cd18a,com_cd20a,com_cd22a,com_cd24a,com_cd27a,...,hh_f08_GRASS,hh_f08_IRON SHEETS,hh_f08_CLAY TILES,hh_f08_CONCRETE,hh_f08_PLASTIC SHEETING,hh_f08_OTHER (SPECIFY),com_cd01_TAR/ASPHALT,com_cd01_GRADED GRAVELED,com_cd01_DIRT ROAD (MAINTAINED),com_cd01_DIRT TRACK
0,-17.095150,35.217213,1437.305054,2.541667,NaN,NaN,NaN,NaN,NaN,NaN,...,0.666667,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-17.092351,35.114643,1197.992310,2.157895,5.0,5.0,5.0,7.0,2.0,4.0,...,0.789474,0.210526,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-17.016698,35.079629,1843.399536,2.333333,60.0,45.0,45.0,60.0,7.0,0.0,...,0.571429,0.428571,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-16.977243,35.205706,1543.134766,2.264706,2.0,2.0,2.0,15.0,0.0,1.0,...,0.705882,0.294118,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-16.956385,35.168967,992.928833,2.045455,18.0,18.0,18.0,40.0,NaN,500.0,...,0.454545,0.545455,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,-9.591378,33.057450,1184.966797,3.500000,2.0,NaN,5.0,5.0,5.0,1.0,...,0.444444,0.555556,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
776,-9.550397,33.291558,990.561401,2.851852,NaN,5.0,5.0,21.0,NaN,0.0,...,0.407407,0.592593,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
777,-9.519230,33.139193,1761.653442,2.971429,NaN,8.0,8.0,41.0,41.0,0.0,...,0.371429,0.628571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
778,-9.507538,33.259649,1525.406860,3.000000,NaN,23.0,23.0,50.0,50.0,0.0,...,0.255814,0.744186,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
# a few columns have a high percentage of NA
cluster_df.isna().sum() / len(cluster_df)

lat_modified                       0.000000
lon_modified                       0.000000
cons                               0.000000
hh_f10                             0.000000
com_cd16                           0.379487
com_cd18a                          0.350000
com_cd20a                          0.257692
com_cd22a                          0.151282
com_cd24a                          0.191026
com_cd27a                          0.032051
com_cd36a                          0.032051
com_cd40a                          0.032051
com_cd49a                          0.603846
com_cd51a                          0.288462
com_cd60a                          0.032051
com_cd67a                          0.078205
com_cd69a                          0.214103
com_cf08a                          0.434615
dist_admarc                        0.000000
dist_agmrkt                        0.000000
dist_auction                       0.000000
dist_boma                          0.000000
dist_borderpost                 

# Modeling

In [19]:
# This is a bunch of code from the Jean et al Github that is modified to work with Python3 and our data

import numpy as np
import pandas as pd
import random
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import sklearn.linear_model as linear_model
import matplotlib.pyplot as plt
from matplotlib.collections import EllipseCollection
import seaborn as sns


def predict_consumption(
    X, y, k=5, k_inner=5, points=10,
        alpha_low=1, alpha_high=5, margin=0.25):
    """
    Plots predicted consumption
    """
    y_hat, r2 = run_cv(X, y, k, k_inner, points, alpha_low, alpha_high)
    return X, y, y_hat, r2


def run_cv(X, y, k, k_inner, points, alpha_low, alpha_high, randomize=False):
    """
    Runs nested cross-validation to make predictions and compute r-squared.
    """
    alphas = np.logspace(alpha_low, alpha_high, points)
    r2s = np.zeros((k,))
    y_hat = np.zeros_like(y)
    kf = KFold(n_splits=k, shuffle=True)
    fold = 0
    for train_idx, test_idx in kf.split(X):
        r2s, y_hat, fold = evaluate_fold(
            X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
            randomize)
    return y_hat, r2s.mean()


def scale_features(X_train, X_test):
    """
    Scales features using StandardScaler.
    """
    X_scaler = StandardScaler(with_mean=True, with_std=False)
    X_train = X_scaler.fit_transform(X_train)
    X_test = X_scaler.transform(X_test)
    return X_train, X_test


def train_and_predict_ridge(alpha, X_train, y_train, X_test):
    """
    Trains ridge model and predicts test set.
    """
    ridge = linear_model.Ridge(alpha)
    ridge.fit(X_train, y_train)
    y_hat = ridge.predict(X_test)
    return y_hat


def predict_inner_test_fold(X, y, y_hat, train_idx, test_idx, alpha):
    """
    Predicts inner test fold.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    X_train, X_test = scale_features(X_train, X_test)
    y_hat[test_idx] = train_and_predict_ridge(alpha, X_train, y_train, X_test)
    return y_hat


def find_best_alpha(X, y, k_inner, alphas):
    """
    Finds the best alpha in an inner CV loop.
    """
    kf = KFold(n_splits=k_inner, shuffle=True)
    best_alpha = 0
    best_r2 = 0
    for idx, alpha in enumerate(alphas):
        y_hat = np.zeros_like(y)
        for train_idx, test_idx in kf.split(X):
            y_hat = predict_inner_test_fold(
                X, y, y_hat, train_idx, test_idx, alpha)
        r2 = stats.pearsonr(y, y_hat)[0] ** 2
        if r2 > best_r2:
            best_alpha = alpha
            best_r2 = r2
    print('best alpha', best_alpha)
    return best_alpha


def evaluate_fold(
    X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
        randomize):
    """
    Evaluates one fold of outer CV.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    if randomize:
        random.shuffle(y_train)
    best_alpha = find_best_alpha(X_train, y_train, k_inner, alphas)
    X_train, X_test = scale_features(X_train, X_test)
    y_test_hat = train_and_predict_ridge(best_alpha, X_train, y_train, X_test)
    r2 = stats.pearsonr(y_test, y_test_hat)[0] ** 2
    r2s[fold] = r2
    y_hat[test_idx] = y_test_hat
    return r2s, y_hat, fold + 1


In [20]:
def nan_handler(df):
    nas = df.isna().sum()
    for c in df:
        if nas[c] > 0:
            df[c] = df[c].fillna(df[c].median())
    return df

In [21]:
cleaned_df = nan_handler(cluster_df)

In [22]:
cleaned_df.head()

,lat_modified,lon_modified,cons,hh_f10,com_cd16,com_cd18a,com_cd20a,com_cd22a,com_cd24a,com_cd27a,...,hh_f08_GRASS,hh_f08_IRON SHEETS,hh_f08_CLAY TILES,hh_f08_CONCRETE,hh_f08_PLASTIC SHEETING,hh_f08_OTHER (SPECIFY),com_cd01_TAR/ASPHALT,com_cd01_GRADED GRAVELED,com_cd01_DIRT ROAD (MAINTAINED),com_cd01_DIRT TRACK
0,-17.095150,35.217213,1437.305054,2.541667,7.0,7.0,7.0,10.0,10.0,2.0,...,0.666667,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-17.092351,35.114643,1197.992310,2.157895,5.0,5.0,5.0,7.0,2.0,4.0,...,0.789474,0.210526,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-17.016698,35.079629,1843.399536,2.333333,60.0,45.0,45.0,60.0,7.0,0.0,...,0.571429,0.428571,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-16.977243,35.205706,1543.134766,2.264706,2.0,2.0,2.0,15.0,0.0,1.0,...,0.705882,0.294118,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-16.956385,35.168967,992.928833,2.045455,18.0,18.0,18.0,40.0,10.0,500.0,...,0.454545,0.545455,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
y = cleaned_df['cons'].values
y_log = np.log(y)

to_drop = ['lat_modified', 'lon_modified', 'cons']
x = cleaned_df.drop(to_drop, axis=1).values


In [32]:
_, _, _, r2 = predict_consumption(x, y_log)
r2

best alpha 10.0
best alpha 10.0
best alpha 27.825594022071243
best alpha 10.0
best alpha 10.0


0.43764427666032796

In [33]:
_, _, _, r2 = predict_consumption(x, y)
r2

best alpha 10.0
best alpha 10.0
best alpha 10.0
best alpha 77.4263682681127
best alpha 10.0


0.3350005189842792

In [34]:
def create_train_valid(x, y, frac=0.7):
    n_train = int(0.7*(len(x)))
    inds = np.arange(len(x))
    train_ind = np.random.choice(inds, n_train, replace=False)
    valid_ind = np.delete(inds, train_ind)

    train_x = x[train_ind]
    valid_x = x[valid_ind]

    train_y = y[train_ind]
    valid_y = y[valid_ind]

    ss = StandardScaler() # standardize features
    train_x = ss.fit_transform(train_x)
    valid_x = ss.transform(valid_x)
    
    return train_x, train_y, valid_x, valid_y

In [35]:
train_x, train_y, valid_x, valid_y = create_train_valid(x, y_log, frac=0.7)

In [36]:
ridge = linear_model.Ridge(alpha=20)
ridge.fit(train_x, train_y)
ridge.score(train_x, train_y)

0.5844605992211681

In [37]:
ridge.score(valid_x, valid_y)

0.42560288011756275

In [38]:
ridge.intercept_

7.580487547339975

In [39]:
ridge.coef_

array([ 0.00760505,  0.00997201,  0.00429109, -0.01419215,  0.0106517 ,
        0.04095905, -0.01800397,  0.0553139 ,  0.01307029, -0.02908114,
       -0.01500005,  0.02137759,  0.01844049, -0.00291938, -0.01070527,
       -0.00611226,  0.00968074, -0.03795993, -0.01015121,  0.04803323,
       -0.01599749, -0.02457293, -0.00412753, -0.04628528, -0.15632615,
       -0.00425367,  0.10847051,  0.04274507, -0.11364201,  0.11158904,
        0.03323608,  0.01261595, -0.01745324,  0.01804436,  0.01134696,
       -0.00107838, -0.04389771, -0.01832556])

In [40]:
df_imps = pd.DataFrame.from_dict({'columns': cleaned_df.drop(to_drop,axis=1).columns,
                                 'imps': ridge.coef_})
df_imps.sort_values('imps', ascending=False, inplace=True)

In [41]:
df_imps.head(10)

,columns,imps
29,hh_f08_IRON SHEETS,0.111589
26,af_bio_16,0.108471
7,com_cd36a,0.055314
19,dist_borderpost,0.048033
27,dist_hh,0.042745
5,com_cd24a,0.040959
30,hh_f08_CLAY TILES,0.033236
11,com_cd60a,0.021378
12,com_cd67a,0.018440
33,hh_f08_OTHER (SPECIFY),0.018044


In [42]:
df_imps.tail(10)

,columns,imps
32,hh_f08_PLASTIC SHEETING,-0.017453
6,com_cd27a,-0.018004
37,com_cd01_DIRT TRACK,-0.018326
21,dist_road,-0.024573
9,com_cd49a,-0.029081
17,dist_auction,-0.037960
36,com_cd01_DIRT ROAD (MAINTAINED),-0.043898
23,af_bio_8,-0.046285
28,hh_f08_GRASS,-0.113642
24,af_bio_12,-0.156326
